In [8]:
import ipywidgets as widgets
from ipywidgets import Layout, Box
from sqlalchemy import create_engine
import pandas as pd
import os
import json

from mastodon import Mastodon



In [9]:
# Search Mastodon

access_token = os.getenv("MASTODON_ACCESS_TOKEN")
api_base_url = os.getenv("MASTODON_API_BASE_URL")

mastodon = Mastodon(access_token=access_token, api_base_url=api_base_url)

search_results = mastodon.search('@tchambers@indieweb.social', result_type='accounts')

In [10]:
search_results

{'accounts': [{'id': 895746,
   'username': 'tchambers',
   'acct': 'tchambers@indieweb.social',
   'display_name': 'Tim Chambers',
   'locked': False,
   'bot': False,
   'discoverable': True,
   'indexable': True,
   'group': False,
   'created_at': datetime.datetime(2019, 8, 30, 0, 0, tzinfo=tzutc()),
   'note': '<p>Technologist, writer, admin of indieweb.social. Fascinated by how new politics impacts technology and vice versa.  <a href="https://indieweb.social/tags/fedi22" class="mention hashtag" rel="nofollow noopener noreferrer" target="_blank">#<span>fedi22</span></a> <a href="https://indieweb.social/tags/indieweb" class="mention hashtag" rel="nofollow noopener noreferrer" target="_blank">#<span>indieweb</span></a> <a href="https://indieweb.social/tags/fediverse" class="mention hashtag" rel="nofollow noopener noreferrer" target="_blank">#<span>fediverse</span></a></p>',
   'url': 'https://indieweb.social/@tchambers',
   'uri': 'https://indieweb.social/users/tchambers',
   'avata

In [53]:

# Create SQLite database engine
engine = create_engine("sqlite:///../toots.db")

# Execute SQL query to fetch the toots data
query = """SELECT *
FROM toots
JOIN algo_tags
  ON toots.id = algo_tags.toot_id
"""
df = pd.read_sql(query, engine)

# Display the dataframe
print(df.head())


   id                      author   
0   1  topintech@flipboard.social  \
1   2                      asymco   
2   3          cubicgarden@mas.to   
3   4            Ten@hachyderm.io   
4   5         danilo@hachyderm.io   

                                             content   
0  <p>A new kind of climate denial has taken over...  \
1  <p>Apple was the number one volume seller in t...   
2  <p>Just spotted another <a href="https://mas.t...   
3                                                      
4  <p>In terms of ~disruption~, I’m struggling to...   

                   created_at   url   
0  2024-01-16 21:06:41.000000  None  \
1  2024-01-16 21:20:37.000000  None   
2  2024-01-16 21:36:49.000000  None   
3  2024-01-16 21:39:19.000000  None   
4  2024-01-16 21:39:58.000000  None   

                                            raw_blob tagger_version   
0  {"id": 111767663802200651, "created_at": "2024...            0.1  \
1  {"id": 111767718325885219, "created_at": "2024...           

In [54]:
# Assuming the dataframe is already defined as 'df'
# Expand the JSON in the 'tags' column
df['tags'] = df['tags_json'].apply(json.loads)
df_expanded = pd.json_normalize(df['tags'])

# Concatenate the expanded columns with the original dataframe
df = pd.concat([df, df_expanded], axis=1)
df = df.sort_values(by='created_at', ascending=False)

# Display the updated dataframe
print(df)


    id                       author   
79  80           Cateo@mstdn.social  \
77  78   Purzelinenlotte@troet.cafe   
72  73            hollywoodreporter   
75  76  matoken@inari.opencocon.org   
76  77      danzillawhite@minds.com   
..  ..                          ...   
4    5          danilo@hachyderm.io   
3    4             Ten@hachyderm.io   
2    3           cubicgarden@mas.to   
1    2                       asymco   
0    1   topintech@flipboard.social   

                                              content   
79  <p>Connections <br>Puzzle #221<br>🟨🟨🟨🟨<br>🟪🟪🟪🟪...  \
77  <p>Umzugsunternehmen ☑️<br>Umzugsdatum ☑️<br>U...   
72  <p>NewsNation Taps Chris Stirewalt for New Sun...   
75  <p>しばらく前に4インチちょいのESP32 な4,5k(jpy)位のE-Ink reade...   
76  This is one big figure. <a href="https://www.m...   
..                                                ...   
4   <p>In terms of ~disruption~, I’m struggling to...   
3                                                       
2   <p>Just spotte

In [43]:
def display_timeline(df):
    # Assuming the dataframe is already defined as 'df'
    for author, content, tags, created_at in zip(df['author'], df['content'], df['tags'], df['created_at']):
        toot_content = widgets.HTML(value=f'<strong>{author}</strong>: {content} <p> <em>{created_at}</em> </p>',
            layout=Layout(border='solid', padding='10px', margin='10px', max_width='50em'))
        tag_widgets = []
        for tag, is_true in tags.items():
            if is_true == True:
                tag_button = widgets.Button(description=tag, button_style='info')
                tag_widgets.append(tag_button)
        tags_box = Box(children=tag_widgets, layout=Layout(display='flex', flex_flow='row wrap'))
        toot_box = Box(children=[toot_content, tags_box], layout=Layout(display='flex', flex_flow='column wrap'))
        display(toot_box)


In [51]:
display_timeline(df.head(10))

Box(children=(HTML(value='<strong>Cateo@mstdn.social</strong>: <p>Connections <br>Puzzle #221<br>🟨🟨🟨🟨<br>🟪🟪🟪🟪<…

Box(children=(HTML(value='<strong>Purzelinenlotte@troet.cafe</strong>: <p>Umzugsunternehmen ☑️<br>Umzugsdatum …

Box(children=(HTML(value='<strong>hollywoodreporter</strong>: <p>NewsNation Taps Chris Stirewalt for New Sunda…

Box(children=(HTML(value='<strong>matoken@inari.opencocon.org</strong>: <p>しばらく前に4インチちょいのESP32 な4,5k(jpy)位のE-I…

Box(children=(HTML(value='<strong>danzillawhite@minds.com</strong>: This is one big figure. <a href="https://w…

Box(children=(HTML(value='<strong>eq@ingen.work</strong>: <p><i><span>\u3000</span>ℹ️<span> </span><i><i><span…

Box(children=(HTML(value='<strong>Benjaminblackoak@sfba.social</strong>: <p>My local library was my refuge dur…

Box(children=(HTML(value='<strong>Djeep@noauthority.social</strong>: <p>💪 </p><p><a href="https://youtube.com/…

Box(children=(HTML(value='<strong>hagen</strong>: <p>hank green just praised a piece of journalism so bad that…

Box(children=(HTML(value='<strong>RobinApple</strong>: <p>Bird populations dropped 30% in last 50 years, but w…

In [55]:
# Subset the dataframe to include only rows where is_art is true
display_timeline(df[df['is_positive_valence'] == True].head(10))

Box(children=(HTML(value='<strong>Cateo@mstdn.social</strong>: <p>Connections <br>Puzzle #221<br>🟨🟨🟨🟨<br>🟪🟪🟪🟪<…

Box(children=(HTML(value='<strong>Purzelinenlotte@troet.cafe</strong>: <p>Umzugsunternehmen ☑️<br>Umzugsdatum …

Box(children=(HTML(value='<strong>hollywoodreporter</strong>: <p>NewsNation Taps Chris Stirewalt for New Sunda…

Box(children=(HTML(value='<strong>matoken@inari.opencocon.org</strong>: <p>しばらく前に4インチちょいのESP32 な4,5k(jpy)位のE-I…

Box(children=(HTML(value='<strong>danzillawhite@minds.com</strong>: This is one big figure. <a href="https://w…

Box(children=(HTML(value='<strong>Benjaminblackoak@sfba.social</strong>: <p>My local library was my refuge dur…

Box(children=(HTML(value='<strong>Djeep@noauthority.social</strong>: <p>💪 </p><p><a href="https://youtube.com/…

Box(children=(HTML(value='<strong>rinsuki@mstdn.rinsuki.net</strong>: <p>なんとなくGo製のバイナリってその性質から逆コンパイル面倒そうだよなーと思…

Box(children=(HTML(value='<strong>dreilandnews</strong>: <p>Lörrach: Rathaus-Sanierung trifft bei Einwohnerver…

Box(children=(HTML(value='<strong>schirdewan@respublicae.eu</strong>: <p>RT by <span class="h-card"><a href="h…

In [56]:
display_timeline(df[df['is_chill'] == True])

Box(children=(HTML(value='<strong>Cateo@mstdn.social</strong>: <p>Connections <br>Puzzle #221<br>🟨🟨🟨🟨<br>🟪🟪🟪🟪<…

Box(children=(HTML(value='<strong>Purzelinenlotte@troet.cafe</strong>: <p>Umzugsunternehmen ☑️<br>Umzugsdatum …

Box(children=(HTML(value='<strong>hollywoodreporter</strong>: <p>NewsNation Taps Chris Stirewalt for New Sunda…

Box(children=(HTML(value='<strong>danzillawhite@minds.com</strong>: This is one big figure. <a href="https://w…

Box(children=(HTML(value='<strong>Djeep@noauthority.social</strong>: <p>💪 </p><p><a href="https://youtube.com/…

...

Box(children=(HTML(value='<strong>kottke@botsin.space</strong>: <p>The Frozen Colors of Winter. Gorgeous photo…

Box(children=(HTML(value='<strong>mike@flipboard.social</strong>: <p>You’re going to love this episode of Dot …

Box(children=(HTML(value='<strong>manton@manton.org</strong>: <p>There’s some interesting stuff <a href="https…

Box(children=(HTML(value='<strong>tchambers@indieweb.social</strong>: <p><span class="h-card" translate="no"><…

Box(children=(HTML(value='<strong>Ten@hachyderm.io</strong>: ', layout=Layout(border_bottom='solid', border_le…

In [57]:
display_timeline(df[df['is_academic'] == True])

Box(children=(HTML(value='<strong>WisdomExplained@mastodonbooks.net</strong>: <p>I shall love and respect, but…

Box(children=(HTML(value='<strong>root42@chaos.social</strong>: <p>My favorite switch type is the DIP switch a…

Box(children=(HTML(value='<strong>poderobrero@rebelbase.site</strong>: Videoteca de pensamiento marxista - Imp…

Box(children=(HTML(value='<strong>riley@social.audiovalentine.com</strong>: <p>I feel like I\'m seeing 自分 quit…

Box(children=(HTML(value='<strong>Teri_Kanefield</strong>: <p>The problem is that we are currently in an infor…

Box(children=(HTML(value='<strong>Teri_Kanefield</strong>: <p>The authors are political science professors at …

Box(children=(HTML(value='<strong>wearenew_public</strong>: <p>Today: Join the Prosocial Design Network for a …